In [4]:
#rules.py

import cirq
import numpy as np
import torch

def rz_rule(simulator):
    """Apply RZ gate to the first qubit."""
    qubits = simulator.qubits
    gate = cirq.rz(np.pi / 2)
    simulator.add_gate(gate, (qubits[0],))
    return simulator

def x_rule(simulator):
    """Apply X gate to the first qubit."""
    qubits = simulator.qubits
    gate = cirq.X
    simulator.add_gate(gate, (qubits[0],))
    return simulator

def cnot_rule(simulator):
    """Apply CNOT gate between the first and second qubits."""
    qubits = simulator.qubits
    gate = cirq.CNOT
    simulator.add_gate(gate, (qubits[0], qubits[1]))
    return simulator

def swap_rule(simulator):
    """Apply SWAP gate between the first and second qubits."""
    qubits = simulator.qubits
    gate = cirq.SWAP
    simulator.add_gate(gate, (qubits[0], qubits[1]))
    return simulator

def commute_rule(simulator):
    """Swap the order of two adjacent gates if commutable."""
    qubits = simulator.qubits
    if len(simulator.circuit) >= 2:
        op1, op2 = list(simulator.circuit.all_operations())[:2]
        simulator.circuit = cirq.Circuit([op2, op1] + list(simulator.circuit.all_operations())[2:])
    return simulator

def cancel_adjacent_rz(simulator):
    """Cancel adjacent RZ gates if their angles sum to zero."""
    new_ops = []
    for op in simulator.circuit.all_operations():
        if not new_ops or not isinstance(op.gate, cirq.ZPowGate):
            new_ops.append(op)
        else:
            prev_op = new_ops.pop()
            if isinstance(prev_op.gate, cirq.ZPowGate):
                angle_sum = prev_op.gate._exponent + op.gate._exponent
                if np.isclose(angle_sum % 2, 0):
                    continue  # Cancel out
                else:
                    new_op = cirq.rz(angle_sum * np.pi)(prev_op.qubits[0])
                    new_ops.append(new_op)
            else:
                new_ops.append(prev_op)
                new_ops.append(op)
    simulator.circuit = cirq.Circuit(new_ops)
    return simulator

RULES = [rz_rule, x_rule, cnot_rule, swap_rule, commute_rule, cancel_adjacent_rz]

In [5]:
# environment.py
class QuantumCircuitSimulator:
    """Quantum Circuit Simulator using Cirq."""
    def __init__(self, n_qubits, n_moments, n_gate_classes):
        self.n_qubits = n_qubits
        self.n_moments = n_moments
        self.n_gate_classes = n_gate_classes
        self.qubits = [cirq.GridQubit(i, 0) for i in range(n_qubits)]
        self.circuit = cirq.Circuit()
        self.reset()

    def reset(self):
        """Reset simulator state."""
        self.circuit = cirq.Circuit()
        return self.get_state()

    def add_gate(self, gate, qubits):
        """Add a gate to the circuit."""
        self.circuit.append(gate(*qubits))
        return self.get_state()

    def get_state(self):
        """Return circuit state as a tensor (4D)。"""
        # 用 4 个通道模拟门操作，假设每个门类型代表一个通道
        state_tensor = np.zeros((self.n_qubits, self.n_moments, self.n_gate_classes), dtype=np.float32)

        # 示例：根据模拟电路的门操作填充状态张量
        for op in self.circuit.all_operations():
            qubits = [q.row for q in op.qubits]
            gate_type = self.get_gate_type(op.gate)
            if gate_type is not None:
                for qubit in qubits:
                    # 使用时间索引和门索引进行填充
                    moment_index = min(self.n_moments - 1, len(self.circuit))
                    state_tensor[qubit, moment_index, gate_type] = 1.0

        return torch.tensor(state_tensor).permute(2, 0, 1).unsqueeze(0)  # [batch, channels, height, width]

    def get_gate_type(self, gate):
        """获取门类型的索引。"""
        gate_types = ['RZ', 'PX', 'CNOT', 'SWAP']
        if isinstance(gate, cirq.ops.ZPowGate):
            return gate_types.index('RZ')
        if isinstance(gate, cirq.ops.XPowGate):
            return gate_types.index('PX')
        if isinstance(gate, cirq.ops.CNotPowGate):
            return gate_types.index('CNOT')
        if isinstance(gate, cirq.ops.SwapPowGate):
            return gate_types.index('SWAP')
        return None

    def apply_rule(self, rule):
        """Apply a transformation rule to the circuit."""
        rule(self)
        reward = self.compute_reward()
        done = self.check_done()
        return self.get_state(), reward, done

    def compute_reward(self):
        """Compute reward based on circuit quality."""
        depth = len(self.circuit)
        gate_count = sum(len(op.qubits) for op in self.circuit.all_operations())
        reward = -depth - 0.2 * gate_count
        return reward

    def check_done(self):
        """Check if optimization is complete."""
        # Implement custom termination criteria...
        return False

class QuantumCircuitEnvironment:
    """RL environment for quantum circuit optimization."""    
    def __init__(self, n_qubits, n_moments, rules, n_gate_classes):
        """
        Initialize the quantum circuit environment.
        
        :param n_qubits: Number of qubits in the circuit.
        :param n_moments: Number of moments (time steps) in the circuit.
        :param rules: List of transformation rules applicable to the circuit.
        :param n_gate_classes: Number of distinct gate classes considered in the simulation.
        """
        self.simulator = QuantumCircuitSimulator(n_qubits, n_moments, n_gate_classes)
        self.rules = rules
        self.reset()

    def reset(self):
        """Reset environment state."""
        self.state = self.simulator.reset()
        self.done = False
        return self.state

    def apply_rule(self, action):
        """Apply circuit transformation rule."""
        rule_index, _ = action  # 解包元组
        if not (0 <= rule_index < len(self.rules)):
            raise IndexError(f"Rule index {rule_index} is out of range. Valid range is [0, {len(self.rules) - 1}].")
        rule = self.rules[rule_index]
        state, reward, done = self.simulator.apply_rule(rule)
        self.done = done
        return state, reward, self.done

class ActionMask:
    """Helper class for masking illegal actions."""
    def __init__(self, n_rules, n_qubits, n_moments):
        self.n_rules = n_rules
        self.n_qubits = n_qubits
        self.n_moments = n_moments

    def mask(self, circuit, gate_classes):
        """Compute the action mask based on the current circuit state."""
        mask = np.ones((self.n_rules, self.n_qubits * self.n_moments), dtype=bool)  # 使用Python内置的bool类型
        # Implement masking logic based on valid circuit transformation rules
        return torch.tensor(mask, dtype=torch.float32)


In [6]:
# 测试规则
if __name__ == '__main__':

    n_qubits = 5
    n_moments = 15
    n_gate_classes = 4

    simulator = QuantumCircuitSimulator(n_qubits, n_moments, n_gate_classes)

    # 测试应用 RZ 规则
    state, reward, done = simulator.apply_rule(rz_rule)
    print(f"State after applying RZ rule: {state.shape}, Reward: {reward}, Done: {done}")

    # 测试应用 CNOT 规则
    state, reward, done = simulator.apply_rule(cnot_rule)
    print(f"State after applying CNOT rule: {state.shape}, Reward: {reward}, Done: {done}")

State after applying RZ rule: torch.Size([1, 4, 5, 15]), Reward: -1.2, Done: False
State after applying CNOT rule: torch.Size([1, 4, 5, 15]), Reward: -2.6, Done: False
